In [4]:
import pandas as pds

Import data

In [31]:
def read_file(fname):
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    
    return pds.DataFrame(content)

In [42]:
train = pds.concat([read_file('x_train.txt'), read_file('y_train.txt')], axis=1)
train.columns = ['data', 'label']

test = pds.concat([read_file('x_test.txt'), read_file('y_test.txt')], axis=1)
test.columns = ['data', 'label']

In [44]:
test.head(10)

,data,label
0,Ne l fin de l seclo XIX l Japon era inda çconh...,mwl
1,Schiedam is gelegen tussen Rotterdam en Vlaard...,nld
2,"ГIурусаз батальонал, гьоркьор гIарадабиги лъун...",ava
3,ರಾಜ್ಯಶಾಸ್ತ್ರದ ಪಿತಾಮಹೆ ಅರಿಸ್ಟಾಟಲ್. ರಾಜ್ಯಶಾಸ್ತ್ರ...,tcy
4,Halukum adalah kelenjar tiroid nang menonjol d...,bjn
5,Тухайн үед дан ганц 20кг-ын Орос баллон хэрэгл...,mon
6,پاپاپلنگي، ايتا شاخه گل محمدي، هجري قمري، كُلش...,glk
7,2007 йисан агьалияр сиягьдиз къачунин малуматр...,lez
8,Оперативните програми (ОП) като форма на финан...,bul
9,"Tī Feigneux ê sì-ûi ū Béthancourt-en-Valois, C...",nan
